In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time as time_lib
from scripts.network import network
from scripts.training import trainModel
from scripts.dynamics import vecField,vecField_np

In [ ]:
torch.manual_seed(1)
np.random.seed(1)
dtype=torch.float32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
system = "SIR"

vecRef = vecField(system=system)

if system=="SIR":
        beta,gamma,N = vecRef.beta,vecRef.gamma,vecRef.N
        t_max = 100.
        num_t = 101
        L = 3
        n_t = 2
        vecRef.dt_fine = 1e-2
elif system=="Brusselator":
        A,B = vecRef.A,vecRef.B 
        t_max = 12.
        num_t = 25
        L = 3
        n_t = 5
        vecRef.dt_fine = t_max/640
else:
        print("Dynamics not implemented")

In [ ]:
if system=="SIR":
        y0 = np.array([0.3,0.5,0.2])
elif system=="Brusselator":
        y0 = np.array([0.,1.])
else:
        print("Dynamics not implemented")

In [ ]:
#Domain details
t0 = 0. #initial time
dt = 1.#t_max / num_t #time step for which we train the network
if system=="SIR":
    lb = 0.
    ub = 1.
    d = 3
elif system=="Brusselator":
    lb = 0.
    ub = 5.
    d = 2

n_train, epochs = 1000, int(1e5)
vec = vecField(system,d)
is_training = input("Do you want to train the network or load a pretrained model? Write y to train it")=="y"

lr = 5e-3

nlayers = 4
hidden_nodes = 10
act = "sin"
dim_t = hidden_nodes
device = 'cpu'
bounds = [lb,ub]

model = network(neurons=hidden_nodes,d=d,dt=dt,act_name=act,nlayers=nlayers,dtype=dtype,system=system) #If you want a different network
model.to(device);

lr = 5e-3

tt = time_lib.time()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
timestamp = time_lib.strftime("%Y%m%d_%H%M%S") 
file_name = f"trained_model_{system}_{timestamp}"
path = f"trainedModels/{file_name}.pt"
Loss = trainModel(model,n_train,y0,dt,t_max,bounds,vec,epochs,device,dtype,optimizer)
torch.save(model.state_dict(), path)
model.eval();

training_time = time_lib.time() - tt

In [ ]:
params = {
    "dt":dt,
    "n_train":n_train,
    "epochs":epochs,
    "nlayers":nlayers,
    "lr":lr,
    "act":act,
    "hidden_nodes":hidden_nodes,
    "dim_t":dim_t,
    "device":device,
    "file_name":file_name,
    "training_time":training_time
}

In [ ]:
import pickle 

with open(f'trainedModels/{file_name}.pkl', 'wb') as f:
    pickle.dump(params, f)

In [ ]:
training_time